# Breadth first search

Breadth first search was added in MADlib 1.12.

In [21]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [22]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [23]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12-dev, git revision: rel/v1.11-29-g8c9b955, cmake configuration time: Thu Jul 13 00:17:54 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create vertex and edge tables

In [85]:
%%sql 
DROP TABLE IF EXISTS vertex, edge;

CREATE TABLE vertex(
        id INTEGER
        );

CREATE TABLE edge(
        src INTEGER,
        dest INTEGER
        );

INSERT INTO vertex VALUES
(0),
(1),
(2),
(3),
(4),
(5),
(6),
(7),
(8),
(9),
(10),
(11);

INSERT INTO edge VALUES
(0, 5),
(1, 0),
(1, 3),
(2, 6),
(3, 4),
(3, 5),
(4, 2),
(8, 9),
(9, 10),
(9, 11),
(10, 8);

SELECT * FROM edge ORDER BY src, dest;

Done.
Done.
Done.
12 rows affected.
11 rows affected.
11 rows affected.


src,dest
0,5
1,0
1,3
2,6
3,4
3,5
4,2
8,9
9,10
9,11


# 2.  Traverse undirected graph from vertex 3

In [86]:
%%sql
DROP TABLE IF EXISTS out, out_summary;

SELECT madlib.graph_bfs(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertix id column (NULL means use default naming)
                         'edge',        -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         3,             -- Source vertex for BFS
                         'out');        -- Output table of nodes reachable from source_vertex
                                        -- Default values used for the other arguments

SELECT * FROM out ORDER BY dist,id;

Done.
1 rows affected.
7 rows affected.


id,dist,parent
3,0,None
1,1,3
4,1,3
5,1,3
0,2,1
2,2,4
6,3,2


In [88]:
%%sql
SELECT * FROM out_summary;

1 rows affected.


vertex_table,vertex_id,edge_table,edge_args,source_vertex,out_table,max_distance,directed,grouping_cols
vertex,NULL,edge,NULL,3,out,None,None,NULL


# 3.  Use max_distance to limit the search distance

In [87]:
%%sql
DROP TABLE IF EXISTS out_max, out_max_summary;

SELECT madlib.graph_bfs(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertix id column (NULL means use default naming)
                         'edge',        -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         3,             -- Source vertex for BFS
                         'out_max',     -- Output table of nodes reachable from source_vertex
                         2);            -- Maximum distance to traverse from source_vertex        
                                        -- Default values used for the other arguments
    
SELECT * FROM out_max ORDER BY dist,id;

Done.
1 rows affected.
6 rows affected.


id,dist,parent
3,0,None
1,1,3
4,1,3
5,1,3
0,2,1
2,2,4


# 4.  Use different column naming

Now let's do an example using different column names in the tables (i.e., not the defaults). Create the vertex and edge tables:

In [89]:
%%sql
DROP TABLE IF EXISTS vertex_alt, edge_alt;
CREATE TABLE vertex_alt AS SELECT id AS v_id FROM vertex;
CREATE TABLE edge_alt AS SELECT src AS n1, dest AS n2 FROM edge;
SELECT * FROM edge_alt ORDER BY n1, n2;

Done.
12 rows affected.
11 rows affected.
11 rows affected.


n1,n2
0,5
1,0
1,3
2,6
3,4
3,5
4,2
8,9
9,10
9,11


# 5. Run BFS from vertex 8

In [90]:
%%sql
DROP TABLE IF EXISTS out_alt, out_alt_summary;

SELECT madlib.graph_bfs(
                         'vertex_alt',                  -- Vertex table
                         'v_id',                        -- Vertex id column (NULL means use default naming)
                         'edge_alt',                    -- Edge table
                         'src=n1, dest=n2',             -- Edge arguments (NULL means use default naming)
                         8,                             -- Source vertex for BFS
                         'out_alt');                    -- Output table of nodes reachable from source_vertex

SELECT * FROM out_alt ORDER BY v_id;

Done.
1 rows affected.
4 rows affected.


v_id,dist,parent
8,0,None
9,1,8
10,1,8
11,2,9


# 6. Directed graph

Now we show an example where the graph is treated as a directed graph.

In [91]:
%%sql
DROP TABLE IF EXISTS out_alt_dir, out_alt_dir_summary;

SELECT madlib.graph_bfs(
                         'vertex_alt',                  -- Vertex table
                         'v_id',                        -- Vertex id column (NULL means use default naming)
                         'edge_alt',                    -- Edge table
                         'src=n1, dest=n2',             -- Edge arguments (NULL means use default naming)
                         8,                             -- Source vertex for BFS
                         'out_alt_dir',                 -- Output table of nodes reachable from source_vertex
                         NULL,                          -- Maximum distance to traverse from source_vertex
                         TRUE);                         -- Flag for specifying directed graph

SELECT * FROM out_alt_dir ORDER BY v_id;

Done.
1 rows affected.
4 rows affected.


v_id,dist,parent
8,0,None
9,1,8
10,2,9
11,2,9


Notice that, with the graph being treated as directed, the parent of v_id=10 is now vertex 9 and not 8 as in the undirected case.

# 7. Grouping

Create a graph with 2 groups.

In [92]:
%%sql
DROP TABLE IF EXISTS edge_gr;

CREATE TABLE edge_gr(
                  g1 INTEGER,
                  g2 TEXT,
                  src INTEGER,
                  dest INTEGER
                );

INSERT INTO edge_gr VALUES
(100, 'a', 0, 5),
(100, 'a', 1, 0),
(100, 'a', 1, 3),
(100, 'a', 2, 6),
(100, 'a', 3, 4),
(100, 'a', 3, 5),
(100, 'a', 4, 2),
(100, 'a', 8, 9),
(100, 'a', 9, 10),
(100, 'a', 9, 11),
(100, 'a', 10, 8),
(202, 'c', 8, 9),
(202, 'c', 9, 10),
(202, 'c', 9, 11),
(202, 'c', 10, 8);

SELECT * FROM edge_gr ORDER BY g1, g2;

Done.
Done.
15 rows affected.
15 rows affected.


g1,g2,src,dest
100,a,1,0
100,a,1,3
100,a,2,6
100,a,3,4
100,a,3,5
100,a,4,2
100,a,8,9
100,a,9,10
100,a,9,11
100,a,10,8


# 8. Run BFS for all groups

In [93]:
%%sql
DROP TABLE IF EXISTS out_gr, out_gr_summary;

SELECT madlib.graph_bfs(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertex id column (NULL means use default naming)
                         'edge_gr',     -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         8,             -- Source vertex for BFS
                         'out_gr',      -- Output table of nodes reachable from source_vertex
                         NULL,          -- Maximum distance to traverse from source_vertex
                         NULL,          -- Flag for specifying directed graph
                         'g1,g2'        -- Grouping columns
);

SELECT * FROM out_gr ORDER BY g1,g2,dist,id;

Done.
1 rows affected.
8 rows affected.


g1,g2,id,dist,parent
100,a,8,0,None
100,a,9,1,8
100,a,10,1,8
100,a,11,2,9
202,c,8,0,None
202,c,9,1,8
202,c,10,1,8
202,c,11,2,9


If source_vertex is not present in a group, then that group will not appear in the output table:

In [94]:
%%sql
DROP TABLE IF EXISTS out_gr, out_gr_summary;

SELECT madlib.graph_bfs(
                         'vertex',      -- Vertex table
                         NULL,          -- Vertex id column (NULL means use default naming)
                         'edge_gr',     -- Edge table
                         NULL,          -- Edge arguments (NULL means use default naming)
                         3,             -- Source vertex for BFS
                         'out_gr',      -- Output table of nodes reachable from source_vertex
                         NULL,          -- Maximum distance to traverse from source_vertex
                         NULL,          -- Flag for specifying directed graph
                         'g1,g2'        -- Grouping columns
);

SELECT * FROM out_gr ORDER BY g1,g2,dist,id;

Done.
1 rows affected.
7 rows affected.


g1,g2,id,dist,parent
100,a,3,0,None
100,a,1,1,3
100,a,4,1,3
100,a,5,1,3
100,a,0,2,1
100,a,2,2,4
100,a,6,3,2
